## Load API KEY

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

# Load variables from the nearest .env file (walking up directories if needed)
load_dotenv(find_dotenv(), override=False)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY not found. Make sure it's set in your .env file.")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not found. Make sure it's set in your .env file.")



## load & split mp3 files

In [ ]:
# 讀取 MP3 檔案
from pydub import AudioSegment


def split_audio(source: AudioSegment, length: int):
    """
    Split an AudioSegment into chunks.

    Parameters:
        source (AudioSegment): the loaded audio file.
        length (int): chunk length in milliseconds (e.g., 60_000 for 1 minute).

    Returns:
        list[AudioSegment]: list of audio chunks.
    """
    chunks = []
    for idx in range(0, len(source), length):
        chunks.append(source[idx:idx + length])
    return chunks


input_mp3 = './sound_files/' + 'my_CH_001.mp3'
if not os.path.isfile(input_mp3):
    raise RuntimeError("MP3檔案不存在")

# 分割mp3檔，設定每幾秒做分割
chunk_length = 60 * 1000 * 10

audio = AudioSegment.from_mp3(input_mp3)
audio_slices = split_audio(source=audio, length=chunk_length)

## process mp3 slices one by one